In [3]:
#trans_temp
#This will only work for Oracle Data Bases
tab_name = input('Enter Table Name:')
sch_name = 'trans_product_gsdb_gsdb'
print('--TRANS TEMP DDL')
# print('SET ANSI_NULLS ON')
# print('GO')
# print('SET QUOTED_IDENTIFIER ON')
# print('GO')
print('CREATE TABLE ['+sch_name+'].['+tab_name+'_init_temp]')
print('(')
print('  [infa_operation_time] [varchar](35) NULL,')
print('  [infa_operation_type] [varchar](1) NULL,')
dict1 = {}
with open('untitled.txt') as inp:
    for line in inp:
        key, val = line.split()
        val = val.replace('NUMBER','varchar](100)')
        val = val.replace('VARCHAR2','varchar]')
        val = val.replace('DATE','varchar](35)')
        dict1[(key)]=val
        res = {key: val for key, val in sorted(dict1.items(), key = lambda ele: ele[0])}
    for key,val in res.items():
        print('  ['+key.lower()+'] ['+val+',')
print('  [ingest_partition] [varchar](100) NULL,')
print('  [ingest_channel] [varchar](100) NULL,')
print('  [file_path] [varchar](100) NULL,')
print('  [root_path] [varchar](100) NULL')
print(')')
print('WITH')
print('(')
print('  DISTRIBUTION = ROUND_ROBIN,')
print('  HEAP')
print(')')
print('GO')


#trans layer
#This will only work for Oracle Data Bases
print('--TRANS DDL')
print('-------------------')
# print('SET ANSI_NULLS ON')
# print('GO')
# print('SET QUOTED_IDENTIFIER ON')
# print('GO')
print('CREATE TABLE ['+sch_name+'].['+tab_name+']')
print('(')
#print('  [infa_operation_time] [varchar](35) NULL,')
dict1 = {}
with open('untitled.txt') as inp:
    for line in inp:
        key, val = line.split()
        val = val.replace('NUMBER','decimal](38,0)')
        val = val.replace('VARCHAR2','varchar]')
        val = val.replace('DATE','datetime2](7)')
        #print('  ['+col_name.lower()+'] ['+dtype+',')
        dict1[(key)]=val
        res = {key: val for key, val in sorted(dict1.items(), key = lambda ele: ele[0])}
    for key,val in res.items():
        print('  ['+key.lower()+'] ['+val+',')
print('  [ingest_partition] [varchar](100) NULL,')
print('  [ingest_channel] [varchar](100) NULL,')
print('  [file_path] [varchar](100) NULL,')
print('  [root_path] [varchar](100) NULL,')
print('  [pipeline_name] [varchar](100) NULL,')
print('  [pipeline_run_id] [varchar](100) NULL,')
print('  [pipeline_trigger_name] [varchar](100) NULL,')
print('  [pipeline_trigger_id] [varchar](100) NULL,')
print('  [pipeline_trigger_type] [varchar](100) NULL,')
print('  [pipeline_trigger_date_time_utc] [datetime2](7) NULL,')
print('  [trans_load_date_time_utc] [datetime2](7) NULL,')
print('  [adle_transaction_code] [char](1) NULL,')
#print('  [infa_sortable_sequence] [varchar](1)')
print('  [hash_key] [varbinary](32) NULL')
print(')')
print('WITH')
print('(')
print('  DISTRIBUTION = HASH([hash_key]),')
print('  CLUSTERED COLUMNSTORE INDEX')
print(')')
print('GO')


#trans_stored_procedure
print('--TRANS STORED PROCEDURE')
print('--------------------------------')
# print('SET ANSI_NULLS ON')
# print('GO')
# print('SET QUOTED_IDENTIFIER ON')
# print('GO')
print("CREATE PROC ["+sch_name+"].["+tab_name+"_proc] @pipeline_name [VARCHAR](100),@pipeline_run_id [VARCHAR](100),@pipeline_trigger_name [VARCHAR](100),@pipeline_trigger_id [VARCHAR](100),@pipeline_trigger_type [VARCHAR](100),@pipeline_trigger_date_time_utc [DATETIME2] AS")
print('BEGIN TRY')
print('--LOAD TYPE: Incremental')
#print("DECLARE @instance_id VARCHAR(4) = '8702';")
print("WITH gen_hashkey as ( ")
print('      SELECT')
dict1 = {}
with open('untitled.txt') as inp:
    for line in inp:
        key, val = line.split()
        val = val.replace('NUMBER','decimal](38,0)')
        val = val.replace('VARCHAR2','varchar]')
        val = val.replace('DATE','datetime2](7)')
#         print('  ['+col_name.lower()+'] ['+dtype+',')
        dict1[(key)]=val
        res = {key: val for key, val in sorted(dict1.items(), key = lambda ele: ele[0])}
for key, val in res.items():
    if val == 'datetime2](7)':
        print("         CAST(STUFF(STUFF(STUFF("+key.lower()+",13,0,':'),11,0,':'),9,0,' ') as ["+val+") AS ["+key.lower()+'],')
    else:
        print('         CAST(['+key.lower()+'] AS ['+val+') AS ['+key.lower()+'],')
print("         CAST(STUFF(STUFF(STUFF(STUFF(replace(ingest_partition,'_',''),15,0,'.'),13,0,':'),11,0,':'),9,0,' ') as DATETIME2(7)) as [ingest_partition],")
print('         [ingest_channel] AS [ingest_channel],')
print('         [file_path] AS [file_path],')
print('         [root_path] AS [root_path],')
print('         CAST([infa_operation_type] as [varchar](1)) as [infa_operation_type],')
print('         CAST(@pipeline_name AS [varchar](100)) AS [pipeline_name] ,')
print('         CAST(@pipeline_run_id AS [varchar](100)) AS [pipeline_run_id],')
print('         CAST(@pipeline_trigger_name AS [varchar](100)) AS [pipeline_trigger_name],')
print('         CAST(@pipeline_trigger_id AS [varchar](100)) AS [pipeline_trigger_id],')
print('         CAST(@pipeline_trigger_type AS [varchar](100)) AS [pipeline_trigger_type],')
print('         CAST(@pipeline_trigger_date_time_utc AS [datetime2](7)) AS [pipeline_trigger_date_time_utc],')
print('         CAST(GETDATE() AS [datetime2](7)) AS [trans_load_date_time_utc],')
print("         'I' as adle_transaction_code,")
# print("         CAST(STUFF(STUFF(STUFF(SUBSTRING(infa_operation_time,1,14),13,0,':'),11,0,':'),9,0,' ') as DATETIME2(7)) as [infa_operation_time],")
# print('         CAST(1 as [varchar](100)) as infa_sortable_sequence,')
print("         CAST(hashbytes('sha2_256',(upper(trim(coalesce([product_status_code], ''))))) as VARBINARY(32)) AS [hash_key] ")
print("      FROM    ["+sch_name+"].["+tab_name+"init_temp]")


print(")")
print(",rn as (")
print("    SELECT  *, ROW_NUMBER() OVER (PARTITION BY hash_key ORDER BY")
print("                CAST([last_update_date] AS [datetime2](0)) DESC) as _ELT_ROWNUMBERED")
print("    FROM    gen_hashkey")
print(")")
print("MERGE INTO    ["+sch_name+"].["+tab_name+"] tgt")
print("USING (")
print("    SELECT  *")
print("    FROM rn")
print("    WHERE _ELT_ROWNUMBERED = 1")
print(") src")
print("ON ( src.[hash_key] = tgt.[hash_key] )")
print("WHEN MATCHED THEN ")
print("UPDATE SET")

with open('untitled.txt') as inp:
    for line in inp:
        key, val = line.split()
        val = val.replace('NUMBER','varchar](100)')
        val = val.replace('VARCHAR2','varchar]')
        val = val.replace('DATE','varchar](35)')
        dict1[(key)]=val
        res = {key: val for key, val in sorted(dict1.items(), key = lambda ele: ele[0])}
    for key,val in res.items():
        print("        [tgt].["+key.lower()+"] = [src].["+key.lower()+"],")        
print("        [tgt].[ingest_partition] = src.[ingest_partition],")
print("        [tgt].[ingest_channel] = src.[ingest_channel],")
print("        [tgt].[file_path] = src.[file_path],")
print("        [tgt].[root_path] = src.[root_path],")
print("        [tgt].[trans_load_date_time_utc] = GETDATE(),")
print("        [tgt].[adle_transaction_code] = src.[infa_operation_type],")
print("        [tgt].[pipeline_name] = @pipeline_name,")
print("        [tgt].[pipeline_run_id] = @pipeline_run_id,")
print("        [tgt].[pipeline_trigger_name] = @pipeline_trigger_name,")
print("        [tgt].[pipeline_trigger_id] = @pipeline_trigger_id,")
print("        [tgt].[pipeline_trigger_type] = @pipeline_trigger_type,")
print("        [tgt].[pipeline_trigger_date_time_utc] = @pipeline_trigger_date_time_utc")
print("WHEN NOT MATCHED THEN ")
print("    INSERT (")
with open('untitled.txt') as inp:
    for line in inp:
        key, val = line.split()
        val = val.replace('NUMBER','varchar](100)')
        val = val.replace('VARCHAR2','varchar]')
        val = val.replace('DATE','varchar](35)')
        dict1[(key)]=val
        res = {key: val for key, val in sorted(dict1.items(), key = lambda ele: ele[0])}
    for key,val in res.items():
        print("        ["+key.lower()+"],")
print("        [hash_key],")
print("        [ingest_partition],")
print("        [ingest_channel],")
print("        [file_path],")
print("        [root_path],")
print("        [trans_load_date_time_utc],")
print("        [adle_transaction_code],")
print("        [pipeline_name],")
print("        [pipeline_run_id],")
print("        [pipeline_trigger_name],")
print("        [pipeline_trigger_id],")
print("        [pipeline_trigger_type],")
print("        [pipeline_trigger_date_time_utc]")
print("    )")
print("    VALUES (")
with open('untitled.txt') as inp:
    for line in inp:
        key, val = line.split()
        val = val.replace('NUMBER','varchar](100)')
        val = val.replace('VARCHAR2','varchar]')
        val = val.replace('DATE','varchar](35)')
        dict1[(key)]=val
        res = {key: val for key, val in sorted(dict1.items(), key = lambda ele: ele[0])}
    for key,val in res.items():
        print("        [src].["+key.lower()+"],")
print("        [src].[hash_key],")
print("        [src].[ingest_partition],")
print("        [src].[ingest_channel],")
print("        [src].[file_path],")
print("        [src].[root_path],")
print("        GETDATE(),")
print("        [src].[infa_operation_type],")
print("        @pipeline_name,")
print("        @pipeline_run_id,")
print("        @pipeline_trigger_name,")
print("        @pipeline_trigger_id,")
print("        @pipeline_trigger_type,")
print("        @pipeline_trigger_date_time_utc")
print("    );")
print('')
print('')
print('END TRY')
print('BEGIN CATCH')
print('    DECLARE @db_name VARCHAR(200),')
print('        @schema_name VARCHAR(200),')
print('        @error_nbr INT,')
print('        @error_severity INT,')
print('        @error_state INT,')
print('        @stored_proc_name VARCHAR(200),')
print('        @error_message VARCHAR(8000),')
print('        @created_date_time DATETIME2')
print('')
print('    SET @db_name=DB_NAME()')
print("    SET @schema_name=SUBSTRING (@pipeline_name, CHARINDEX('2', @pipeline_name) + 1, LEN(@pipeline_name) - CHARINDEX('2', @pipeline_name) - 3 )")
print('    SET @error_nbr=ERROR_NUMBER()')
print('    SET @error_severity=ERROR_SEVERITY()')
print('    SET @error_state=ERROR_STATE()')
print('    SET @stored_proc_name=ERROR_PROCEDURE()')
print('    SET @error_message=ERROR_MESSAGE()')
print('    SET @created_date_time=GETDATE()')
print('')
print('    EXECUTE [adle_platform_orchestration].[elt_error_log_proc]')
print('        @db_name,')
print("        'ERROR',")
print('        @schema_name,')
print('        @error_nbr,')
print('        @error_severity,')
print('        @error_state,')
print('        @stored_proc_name,')
print("        'PROC',")
print('        @error_message,')
print('        @created_date_time,')
print('        @pipeline_name,')
print('        @pipeline_run_id,')
print('        @pipeline_trigger_name,')
print('        @pipeline_trigger_id,')
print('        @pipeline_trigger_type,')
print('        @pipeline_trigger_date_time_utc')
print('        ;')
print('    THROW;')
print('END CATCH')
print('')
print(';')
print('GO')
print('')


#table config
print('--TABLE CONFIG')
print("insert into adle_platform_orchestration.elt_table_config (datasubject,classification,sourcesystem,sourceschema,tablename,ingest_channel,file_type,ingest_partition,layer,container,stg_file_type,sequence_nbr,stg_file_format,cycle,tgt_file_format)")
print("values('manufacturing','confidential','erp','infx'"+tab_name+",'fts-iics','parquet','19000101_010000','trans','stg-dev','parquet',10,'parquet_none','daily','same_as_source');")


Enter Table Name:product_royalty_supplier
--TRANS TEMP DDL
CREATE TABLE [trans_product_gsdb_gsdb].[product_royalty_supplier_init_temp]
(
  [infa_operation_time] [varchar](35) NULL,
  [infa_operation_type] [varchar](1) NULL,
  [contract_expiration_date] [varchar](35),
  [create_id] [varchar](30),
  [create_timestamp] [varchar](35),
  [last_update_id] [varchar](30),
  [last_update_timestamp] [varchar](35),
  [product_detail_key] [varchar](100),
  [royalty_agreement_number] [varchar](60),
  [royalty_supplier_code] [varchar](10),
  [supplier_part_description] [varchar](120),
  [supplier_part_number] [varchar](60),
  [ingest_partition] [varchar](100) NULL,
  [ingest_channel] [varchar](100) NULL,
  [file_path] [varchar](100) NULL,
  [root_path] [varchar](100) NULL
)
WITH
(
  DISTRIBUTION = ROUND_ROBIN,
  HEAP
)
GO
--TRANS DDL
-------------------
CREATE TABLE [trans_product_gsdb_gsdb].[product_royalty_supplier]
(
  [contract_expiration_date] [datetime2](7),
  [create_id] [varchar](30),
  [cre